# LSPIV - Part 2: PIV processing
*GH Gerritsen*
___

After acquiring the individual frames, velocity fields are determined using the `OpenPIV` library [1]. OpenPIV is still in its developing fase and new features to the package can be are added. The coding used in this notebook is based on OpenPIV `v0.21.3`. 

The PIV process in its schematic form is visualised in the figure below. The imagery is divided into gridcells with a set size and overlap. The similarity of the grid cells in two sequential images -- based on the presence of seeds in the form of e.g. debris, leaves, or airbubbles -- are used to estimate displacements. For each image pair a velocity field can be determined.

<img src='LSPIV visual.jpg' width="750">

This notebook uses the frames from the folder `frames`, and saves for each frame pair a .txt file in the folder `files`, which will be created if non-existent.

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from openpiv import tools

import pivprocessing as pivpro

In [2]:
dir_frames = 'frames'
dir_saves = 'files'

In [3]:
if not os.path.exists(dir_saves):
    os.mkdir(dir_saves)

For the PIV process, several parameters have to be determined. These parameters are:

- `windowsize`: size of the individual (square) grid cells in pixels.
- `searchsize`: size of the search area in pixels in which the location with the highest similarity is found. In v0.21.3 the search area has to be the same as the windowsize. In later versions the possibility is provided to adjust the search area to other (non-square) dimensions.
- `overlap`: overlap over the grid cells in pixels.
- `frame_rate`: frame rate of the video in frames per second (fps).
- `scaling_factor`: amount of pixels per meter (PPM in the previous notebook).
- `threshold`: threshold for which signal-to-noise ratio results are filtered

In [4]:
# set-up parameters
winsize = 60             # 60 centimetres if PPM = 100
searchsize = winsize     # same as winsize in v0.21.3
overlap = 30             # overlap of grid cells
frame_rate = 25          # frame rate video
scaling_factor = 100     # pixels per meter (PPM)
threshold = 1.3          # signal-to-noise ratio at which results are filtered out

After setting up the parameters, the PIV processing can be applied. The output of this process are `.txt` files containing for each grid cell the following characteristics:

- x-coordinates of the centre of the grid cell,
- y-coordinates of the centre of the grid cell,
- x-component of the found surface flow velocity,
- y-component of the found surface flow velocities,
- signal-to-noise ratio result used for of the similarity process.

In [5]:
# import the names and number of frames
frames = os.listdir(dir_frames)
N = len(frames)-1

for n in tqdm(range(N)):
    # define the two sequential frames which are used to estimate the velocity field
    frame_0 = tools.imread(os.path.join(dir_frames,str(frames[n])))
    frame_1 = tools.imread(os.path.join(dir_frames,str(frames[n+1])))

    # PIV processing
    x, y, u, v, sig2noise = pivpro.PIV(frame_0, frame_1, winsize, searchsize,
                                       overlap, frame_rate, scaling_factor,
                                       threshold, output='fil')

    # save results in a text file
    tools.save(x,
               y,
               u,
               v,
               sig2noise,
               os.path.join(dir_saves, str(format(n).zfill(6))+'.txt'))

100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [02:12<00:00,  1.12s/it]


___
[1] OpenPIV documentation - https://openpiv.readthedocs.io/en/latest/index.html